# HW8 Implementation

In [2]:
import numpy as np

In [3]:
P = np.array([
    [0, 3/4, 0, 1/4, 0],
    [3/4, 0, 0, 1/4, 0],
    [0, 1/2, 1/2, 0, 0],
    [0, 0, 0, 0, 1],
    [0, 1/2, 1/2, 0, 0]
])

# Calculating P^2 and P^3
P_squared = np.linalg.matrix_power(P, 2)
P_cubed = np.linalg.matrix_power(P, 3)

In [8]:
print(P_cubed)

[[0.       0.546875 0.125    0.140625 0.1875  ]
 [0.421875 0.125    0.125    0.140625 0.1875  ]
 [0.1875   0.40625  0.125    0.15625  0.125   ]
 [0.375    0.25     0.25     0.125    0.      ]
 [0.1875   0.40625  0.125    0.15625  0.125   ]]


# Problem 7 Implementation

Enyan is playing the mobile game **Clash of Probability**. For its $10$ year anniversary promotion, **Clash of Probability** has a special event where you can win one of its **legendary** characters ``Thousand Feather Sapphire Phoenix". The rules of the event is as follows:

- There are $9$ cards $C_1, ..., C_9$ on the screen facing down.
- Each day, the game will randomly flip a card with equal probability.
- A card that has been flipped may be flipped back. For example, if $C_1$ is flipped on the first day, it may be flipped back on the second day.
- Your goal is to flip all $9$ cards over facing up.

Enyan wants to know the approximate expected number of days to obtain the "Thousand Feather Sapphire Phoenix". 

(a) This problem may be modeled as a Markov chain with one absorbing state (when all the cards are flipped over facing up). Write a program that computes the state transition matrix $M$ of this Markov chain in canonical form and explain your thought process. We say the matrix $M$ is in canonical form if the rows and columns of $M$ are rearranged so that the absorbing state is listed last:
$$M = 
\left(\begin{array}{@{}c|c@{}}
  \begin{matrix}
  Q
  \end{matrix}
  & R  \\
\hline
  0\ ...\ 0 & 1
\end{array}\right)
$$

**Answer:** There are in total $2^9=512$ states in this Markov process. We let the absorbing state (with all cards facing up) to be the $512-th$ state.

For each state $i \neq 512$, it can transit to $9$ other states $j$, with equal probability $1/9$. If $j \neq 512$, we also have $M_{ji}=1/9$. The matrix $Q$ is therefore symmetric. For vector matrix $R$, it will only have $9$ non-zero entries, with each having probaility $1/9$. We now generate the transition matrix $M$ with codes.

In [13]:
import numpy as np

def generate_state_transitions():
    """
    Generate the state transitions for the Markov chain model of the game.
    Each state is represented as an integer where the binary representation
    corresponds to the cards' orientation (0 for down, 1 for up).
    Ignore the absorbing state at this moment
    """
    num_cards = 9
    num_states = 2 ** num_cards  # Total number of states

    # Initialize the transition matrix with zeros
    transition_matrix = np.zeros((num_states, num_states))

    for state in range(num_states):
        # For each state, iterate through all cards and flip them
        for card in range(num_cards):
            # Calculate the new state after flipping the card
            # state is automatically converted to binary
            # 1 << card (shift) gives the binary form in exactly the card position
            # ^ is the XOR operator, which flips the specific bit (card)
            new_state = state ^ (1 << card)
            transition_matrix[state, new_state] += 1 / num_cards
    
    # absorbing state
    for state in range(num_states):
        if state==num_states-1:
            transition_matrix[num_states-1, state] = 1
        else:
            transition_matrix[num_states-1, state] = 0

    return transition_matrix

# Generate the state transition matrix
transition_matrix = generate_state_transitions()

In [15]:
transition_matrix

array([[0.        , 0.11111111, 0.11111111, ..., 0.        , 0.        ,
        0.        ],
       [0.11111111, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.11111111, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.11111111],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.11111111],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

(b) Let $I_n$ be the identity matrix with the same dimension as $Q$. The matrix $F = (I_n - Q)^{-1}$ called the fundamental matrix of this Markov Chain. Following the previous step, compute the matrix $F$.

In [17]:
from numpy.linalg import inv

Q = transition_matrix[:-1, :-1]
R = transition_matrix[-1, :]

# Identity matrix of the same dimension as Q
I_n = np.identity(Q.shape[0])

# Compute the fundamental matrix F
F = inv(I_n - Q)

In [18]:
F

array([[2.37142857, 1.37142857, 1.37142857, ..., 1.125     , 1.        ,
        1.        ],
       [1.37142857, 2.36752232, 1.24691685, ..., 1.12060547, 1.00048828,
        0.99609375],
       [1.37142857, 1.24691685, 2.36752232, ..., 1.12060547, 0.99609375,
        1.00048828],
       ...,
       [1.125     , 1.12060547, 1.12060547, ..., 2.24121094, 1.12060547,
        1.12060547],
       [1.        , 1.00048828, 0.99609375, ..., 1.12060547, 1.99609375,
        0.87548828],
       [1.        , 0.99609375, 1.00048828, ..., 1.12060547, 0.87548828,
        1.99609375]])

(c) The expected number of steps to reach the absorbing state from transient state $i$ (indexed by the order in your matrix) is exactly the $i$-th entry of the vector
$$F \cdot \mathbb{1}$$
where $\mathbb{1}$ is the constant vector with entry $1$ of the appropriate dimension. What is the expected number of days it will take for Enyan to win the ``Thousand Feather Sapphire Phoenix"?

In [19]:
ones_vector = np.ones(Q.shape[0])

# Compute the expected number of steps to reach the absorbing state
expected_steps = F.dot(ones_vector)

In [21]:
print('Expected number of days for Eyan to win the game is:')
expected_steps[0]

Expected number of days for Eyan to win the game is:


607.0857142857209

(d) Suppose now **Clash of Probability** replaces $9$ cards with $100$ cards, what is the expected number now?

In [22]:
def calculate_expected_days(num_cards):
    """
    Generate the state transitions for the Markov chain model of the game.
    Each state is represented as an integer where the binary representation
    corresponds to the cards' orientation (0 for down, 1 for up).
    Ignore the absorbing state at this moment
    """
    num_states = 2 ** num_cards  # Total number of states

    # Initialize the transition matrix with zeros
    transition_matrix = np.zeros((num_states, num_states))

    for state in range(num_states):
        # For each state, iterate through all cards and flip them
        for card in range(num_cards):
            # Calculate the new state after flipping the card
            # state is automatically converted to binary
            # 1 << card (shift) gives the binary form in exactly the card position
            # ^ is the XOR operator, which flips the specific bit (card)
            new_state = state ^ (1 << card)
            transition_matrix[state, new_state] += 1 / num_cards
    
    # absorbing state
    for state in range(num_states):
        if state==num_states-1:
            transition_matrix[num_states-1, state] = 1
        else:
            transition_matrix[num_states-1, state] = 0
            
    Q = transition_matrix[:-1, :-1]
    R = transition_matrix[-1, :]
    
    # Identity matrix of the same dimension as Q
    I_n = np.identity(Q.shape[0])
    
    # Compute the fundamental matrix F
    F = inv(I_n - Q)
    ones_vector = np.ones(Q.shape[0])

    # Compute the expected number of steps to reach the absorbing state
    expected_steps = F.dot(ones_vector)
        
    print(f'Expected number of days for Eyan to win the game with {num_cards} cards is {expected_steps[0]}:')
    
    return expected_steps

In [24]:
# MAXIMUM dimension of matrix exceeds
# answer = calculate_expected_days(100)